In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm


REBUILD_DATA = True
class DogsVSCats():
    IMG_SIZE=50
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"
    LABELS={CATS: 0 , DOGS: 1}
    training_data=[]
    catcount=0
    dogcount=0
    def make_training_data(self):
        for label in self.LABELS:#iterating over the directories
            print(label)
            for f in tqdm(os.listdir(label)):#iterating over images in those dir f=file name
                try:
                    path=os.path.join(label,f)
                    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                    img=cv2.resize(img,(self.IMG_SIZE,self.IMG_SIZE))
                    self.training_data.append([np.array(img),np.eye(2)[self.LABELS[label]]])
                
                    if label==self.CATS:
                        self.catcount+=1
                    if label==self.DOGS:
                        self.dogcount+=1
                except Exception as e :
                    pass
        np.random.shuffle(self.training_data)
        np.save("training_data.npy",self.training_data)
        print("Cats:",self.catcount)
        print("Dogs:",self.dogcount)
if REBUILD_DATA:
    dogsvscats=DogsVSCats()
    dogsvscats.make_training_data()
training_data=np.load("training_data.npy",allow_pickle=True)
                
    

PetImages/Cat


100%|███████████████████████████████████████████████████████████████████████████| 12501/12501 [01:28<00:00, 142.05it/s]


PetImages/Dog


100%|███████████████████████████████████████████████████████████████████████████| 12501/12501 [01:35<00:00, 130.26it/s]
C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\lib\npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Cats: 12476
Dogs: 12470


In [32]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,32,5)
        self.conv2=nn.Conv2d(32,64,5)
        self.conv3=nn.Conv2d(64,128,5)
        self.fc1=nn.Linear(128*2*2,512)
        self.fc2=nn.Linear(512,2)  
    def convs(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        
        return x
    def forward(self,x):
        x=self.convs(x)
        x=x.view(-1,128*2*2)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.softmax(x,dim=1)
net = Net()
        

In [33]:
import torch.optim as optim
#                    hon ktabna hek laan for ex bel transfer learning iza
#                    ekhdin pretrained model w bdna nestaamlo to predict
#                    shi tene fina ma naadil b awal kam layer laano ha 
#                    ykouno ktir mneh bas lezim naadil ekhir kam layer
#                    ta yaatuna  l output l bdna yeha 
#                    ***************
optimizer=optim.Adam(net.parameters(),lr=0.001)
loss_function=nn.MSELoss()
X=torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X=X/255.0
y=torch.Tensor([i[1] for i in training_data])
VAL_PCT=0.1
val_size=int(len(X)*VAL_PCT)
print(val_size)

2494


In [50]:
train_X=X[:-val_size]
train_y=y[:-val_size]
test_X=X[-val_size:]
test_y=y[-val_size:]
print(len(train_X))
print(len(test_X))

22452
2494


In [66]:
BATCH_SIZE=100
EPOCHS=3
for epoch in range(EPOCHS):
    for i in tqdm(range(0,len(train_X),BATCH_SIZE)):
        batch_X=train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
        batch_y=train_y[i:i+BATCH_SIZE]
        net.zero_grad()#when we are training our model we set some unnecessary info = 0 to finsih faster
        outputs=net(batch_X)
        loss =loss_function(outputs,batch_y)#laano to calculate the loss , our vector isnt a one hot vector(ex:[0,0,1])
        loss.backward()#In fitting a neural network, backpropagation computes the gradient of the loss function with
        #respect to the weights of the network for a single input–output example, and does so efficiently,
        # unlike a naive direct computation of the gradient with respect to each weight individually.
        optimizer.step()#this adjust the weights
print(loss)

100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [00:57<00:00,  3.89it/s]

tensor(0.1545, grad_fn=<MseLossBackward0>)


In [67]:
import matplotlib.pyplot as plt
print(test_y[1])
print(net(test_X[1].view(-1,1,50,50))[0])
print(torch.argmax(net(test_X[1].view(-1,1,50,50))[0]))

tensor([0., 1.])
tensor([0.4084, 0.5916], grad_fn=<SelectBackward0>)
tensor(1)


In [68]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class=torch.argmax(test_y[i])
        net_out=net(test_X[i].view(-1,1,50,50))[0]
        predicted_class=torch.argmax(net_out)
        if predicted_class == real_class:
            correct+=1
        total+=1
print("Accuracy",round(correct/total,3))

100%|█████████████████████████████████████████████████████████████████████████████| 2494/2494 [00:03<00:00, 828.52it/s]

Accuracy 0.705
